In [ ]:
PATHS MIGHT NEED TO BE FIXED TO RUN THESE CELLS

In [11]:
# Used dataset : https://www.kaggle.com/datasets/joebeachcapital/30000-spotify-songs/data
# https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs
import pandas as pd
import numpy as np
import pickle
import torch
import ast

from sentence_transformers import SentenceTransformer

print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))

model = SentenceTransformer("all-MiniLM-L6-v2")

df = pd.read_csv("./dataset/millionSongs.csv")

df["artists"] = df["artists"].apply(ast.literal_eval)

def build_semantic_text(row):
    artists = ", ".join(row["artists"])
    return (
        f"Song: {row['name']} by {artists}. "
        f"Album: {row['album']}. Released in {row['year']} on {row['release_date']}. "
        f"This track is characterized by danceability {row['danceability']}, energy {row['energy']}, "
        f"valence {row['valence']}, acousticness {row['acousticness']}, instrumentalness {row['instrumentalness']}, "
        f"speechiness {row['speechiness']}, liveness {row['liveness']}, loudness {row['loudness']}, "
        f"and tempo {row['tempo']} BPM. "
        f"It has a key of {row['key']}, mode {row['mode']}, and a time signature of {row['time_signature']}. "
        f"Duration: {row['duration_ms']} milliseconds."
    )

df["semantic_text"] = df.apply(build_semantic_text, axis=1)

texts = df["semantic_text"].tolist()

embeddings = model.encode(texts, batch_size=128, device="cuda", show_progress_bar=True)

# df["embedding"] = df["semantic_text"].apply(lambda x: model.encode(x).tolist())

df["embedding"] = embeddings.tolist()

with open("./million_embeddings.pkl", "wb") as f:
    pickle.dump(df, f)


C:\Users\Admin\Documents\projects\cli-tool\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
NVIDIA GeForce RTX 4070


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9407/9407 [09:56<00:00, 15.78it/s]


In [ ]:
import torch

from sentence_transformers import SentenceTransformer

print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))

model = SentenceTransformer("all-MiniLM-L6-v2")

In [12]:
# import pickle

# with open("./million_embeddings.pkl", "rb") as f:
#     df = pickle.load(f)

embeddings = df["embedding"]

#embeddings_np = np.array(embeddings)  # shape: (num_songs, 384) this is giving 1D arrray
embeddings_np = np.stack(df["embedding"].values).astype("float32")  # shape: (num_songs, 384)
print(embeddings_np.shape)

(1204025, 384)


In [24]:
import faiss

d = embeddings_np.shape[1]  # embedding dimension (384)
index = faiss.IndexFlatIP(d)

faiss.normalize_L2(embeddings_np)

index.add(embeddings_np)

print("Number of vectors in index:", index.ntotal)
faiss.write_index(index, "songs.index")

Number of vectors in index: 1204025


In [1]:
index = faiss.read_index("./songs.index")

q = "The Young God Speaks"
q_emb = model.encode(q).astype("float32").reshape(1, -1)

faiss.normalize_L2(q_emb)

score, index = index.search(q_emb, 10)

for idx, scr in zip(index[0], score[0]):
    print(f"Song: {df['name'].iloc[idx]} : {scr}")

NameError: name 'faiss' is not defined

In [ ]:
# This works but is slow af for data with millions of items
import numpy as np
import pandas as pd
import pickle

def cosine_similarity_np(A, B):
    A = np.array(A)
    B = np.array(B)
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

q = "pop"
q_emb = model.encode(q)
q_emb = q_emb.reshape(1, -1)          # shape: (1, 384)

similarities = (embeddings_np @ q_emb.T).flatten()  # dot product
similarities /= np.linalg.norm(embeddings_np, axis=1)
similarities /= np.linalg.norm(q_emb)

top_idx = np.argsort(-similarities)[:10]

for i in top_idx:
    print(f"{df['name'][i]}: {similarities[i]:.4f}")


In [ ]:
# --- Slowest approch below ---
    
# similarities = [cosine_similarity_np(q_emb, song_emb) for song_emb in embeddings]
    
# songs = df["name"]

# results = list(zip(songs, similarities))

# # Sort by similarity descending
# results = sorted(results, key=lambda x: x[1], reverse=True)

# # Top 10 similar songs
# top_10 = results[:10]

# for song, score in top_10:
#     print(f"{song}: {score:.4f}")

In [2]:
import gc

# del df
# del texts
# del embeddings

gc.collect()

1074